In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install neurokit2
!pip install biosppy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 689.2/689.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import matplotlib.pyplot as plt
import scipy.io as sio
import pandas as pd
import biosppy.signals.ecg as ecg
import numpy as np
import neurokit2 as nk
from scipy.signal import butter, filtfilt
from scipy import signal

path = "/content/drive/MyDrive/DREAMER.mat"
raw = sio.loadmat(path)


In [ ]:
#frequency domain
def preprocess_EEG(raw, feature):
    overall = signal.firwin(9, [0.0625, 0.46875], window="hamming")
    theta = signal.firwin(9, [4, 8], fs=250, window="hamming")
    alpha = signal.firwin(9, [8, 13], fs=250, window="hamming")
    beta = signal.firwin(9, [13, 30], fs=250, window="hamming")
    delta = signal.firwin(9, [0.5, 4], fs=250, window="hamming")  # Add delta band filter
    gamma = signal.firwin(9, [30, 100], fs=250, window="hamming")  # Add gamma band filter

    filt_data = signal.filtfilt(overall, 1, raw)
    filt_theta = signal.filtfilt(theta, 1, filt_data)
    filt_alpha = signal.filtfilt(alpha, 1, filt_data)
    filt_beta = signal.filtfilt(beta, 1, filt_data)
    filt_delta = signal.filtfilt(delta, 1, filt_data)  # Apply delta filter
    filt_gamma = signal.filtfilt(gamma, 1, filt_data)  # Apply gamma filter

    ftheta, psdtheta = signal.welch(filt_theta, fs=128, nperseg=256)
    falpha, psdalpha = signal.welch(filt_alpha, fs=128, nperseg=256)
    fbeta, psdbeta = signal.welch(filt_beta, fs=128, nperseg=256)
    fdelta, psddelta = signal.welch(filt_delta, fs=128, nperseg=256)  # Compute PSD for delta band
    fgamma, psdgamma = signal.welch(filt_gamma, fs=128, nperseg=256)  # Compute PSD for gamma band

    feature.append(np.mean(psdtheta))
    feature.append(np.mean(psdalpha))
    feature.append(np.mean(psdbeta))
    feature.append(np.mean(psddelta))  # Append mean PSD for delta band
    feature.append(np.mean(psdgamma))  # Append mean PSD for gamma band
    return feature

def feat_extract_EEG(raw):
    EEG_tmp = np.zeros((23, 18, 70))  # Update the shape to include delta and gamma bands 14*5=70
    for participant in range(0, 23):
        for video in range(0, 18):
            for i in range(0, 14):
                B, S = [], []
                basl = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["baseline"][0, 0][video, 0][:, i])
                stim = (raw["DREAMER"][0, 0]["Data"]
                        [0, participant]["EEG"][0, 0]
                        ["stimuli"][0, 0][video, 0][:, i])
                B = preprocess_EEG(basl, B)
                S = preprocess_EEG(stim, S)
                Extrod = np.divide(S, B)
                EEG_tmp[participant, video, 5*i] = Extrod[0]
                EEG_tmp[participant, video, 5*i+1] = Extrod[1]
                EEG_tmp[participant, video, 5*i+2] = Extrod[2]
                EEG_tmp[participant, video, 5*i+3] = Extrod[3]
                EEG_tmp[participant, video, 5*i+4] = Extrod[4]

    col = []
    for i in range(0, 14):
        col.append("psdtheta_"+str(i + 1)+"_un")
        col.append("psdalpha_"+str(i + 1)+"_un")
        col.append("psdbeta_"+str(i + 1)+"_un")
        col.append("psddelta_" + str(i + 1) + "_un")  # Add columns for delta band
        col.append("psdgamma_" + str(i + 1) + "_un")  # Add columns for gamma band

    data_EEG = pd.DataFrame(EEG_tmp.reshape((23 * 18,
                                              EEG_tmp.shape[2])), columns=col)
    scaler = StandardScaler()
    for i in range(len(col)):
        data_EEG[col[i][:-3]] = scaler.fit_transform(data_EEG[[col[i]]])
    data_EEG.drop(col, axis=1, inplace=True)
    return data_EEG



def participant_affective(raw):
    a = np.zeros((23, 18, 7), dtype=object)
    for participant in range(0, 23):
        for video in range(0, 18):
            a[participant, video, 0] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Age"][0][0][0])

            a[participant, video, 1] = (raw["DREAMER"][0, 0]["Data"]
                                        [0, participant]["Gender"][0][0][0])

            a[participant, video, 2] = int(participant+1)

            a[participant, video, 3] = int(video+1)

            a[participant, video, 4] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreValence"]
                                           [0, 0][video, 0])
            a[participant, video, 5] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreArousal"]
                                           [0, 0][video, 0])
            a[participant, video, 6] = int(raw["DREAMER"][0, 0]["Data"]
                                           [0, participant]["ScoreDominance"]
                                           [0, 0][video, 0])
    b = pd.DataFrame(a.reshape((23*18, a.shape[2])),
                     columns=["age", "gender", "participant",
                              "video", "valence", "arousal", "dominance"])
    return b

df_features = feat_extract_EEG(raw)
df_participant_affective = participant_affective(raw)

df_participant_affective["valence"] = (df_participant_affective
                                       ["valence"].astype(int))
df_participant_affective["arousal"] = (df_participant_affective
                                       ["arousal"].astype(int))
df_participant_affective["dominance"] = (df_participant_affective
                                         ["dominance"].astype(int))
df = pd.concat([df_features, df_participant_affective], axis=1)

data = df.copy()
data

,psdtheta_1,psdalpha_1,psdbeta_1,psddelta_1,psdgamma_1,psdtheta_2,psdalpha_2,psdbeta_2,psddelta_2,psdgamma_2,...,psdbeta_14,psddelta_14,psdgamma_14,age,gender,participant,video,valence,arousal,dominance
0,-0.150843,-0.150624,-0.166261,-0.151180,-0.243280,-0.145117,-0.143640,-0.119170,-0.146081,-0.184538,...,-0.204449,-0.195448,-0.210802,22,male,1,1,4,3,2
1,-0.128521,-0.129717,-0.155580,-0.127869,-0.025549,-0.105668,-0.108871,-0.168438,-0.103923,-0.033527,...,-0.166154,-0.152931,-0.171532,22,male,1,2,3,3,1
2,-0.103209,-0.103515,-0.122637,-0.103177,-0.176490,-0.109388,-0.111235,-0.140875,-0.108344,-0.111861,...,-0.051510,-0.046362,-0.092479,22,male,1,3,5,4,4
3,-0.181723,-0.184244,-0.230825,-0.180332,-0.178705,-0.113030,-0.114522,-0.139100,-0.112202,-0.126508,...,-0.153138,-0.150298,-0.148628,22,male,1,4,4,3,2
4,-0.043894,-0.042331,-0.032673,-0.044931,-0.149471,-0.148676,-0.152181,-0.201394,-0.146637,-0.124524,...,0.108396,0.073444,-0.135465,22,male,1,5,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,-0.259796,-0.264659,-0.351959,-0.257169,-0.208289,-0.215325,-0.223008,-0.347720,-0.211023,-0.167047,...,-0.049186,-0.053316,-0.117128,25,male,23,14,2,2,2
410,-0.220586,-0.227271,-0.333407,-0.216912,-0.169864,-0.083146,-0.084745,-0.115249,-0.082358,-0.096989,...,0.012865,0.074680,0.144364,25,male,23,15,2,2,2
411,-0.054579,-0.056091,-0.082380,-0.053773,-0.129757,0.014625,0.014887,0.025082,0.014500,0.006592,...,-0.061149,-0.023717,-0.015227,25,male,23,16,3,3,2
412,-0.242060,-0.244057,-0.288320,-0.241022,-0.216108,-0.134908,-0.139699,-0.208673,-0.132119,-0.089472,...,-0.121816,-0.119211,-0.159887,25,male,23,17,2,2,4


In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your DataFrame with the necessary columns

# Define a function to convert values to binary
def to_binary(value):
    return 1 if value > 1 else 0

print("Unique values before transformation:")
print("Valence:", df["valence"].unique())
print("Arousal:", df["arousal"].unique())
print("Dominance:", df["dominance"].unique())

# Apply this function to the "valence", "arousal", and "dominance" columns
data["valence"] = df["valence"].apply(to_binary)
data["arousal"] = df["arousal"].apply(to_binary)
data["dominance"] = df["dominance"].apply(to_binary)

print("Unique values after transformation:")
print("Valence:", data["valence"].unique())
print("Arousal:", data["arousal"].unique())
print("Dominance:", data["dominance"].unique())
# Verify the changes
print(data[["valence", "arousal", "dominance"]].head())

data

Unique values before transformation:
Valence: [4 3 5 1 2]
Arousal: [3 4 2 1 5]
Dominance: [2 1 4 5 3]
Unique values after transformation:
Valence: [2 1 0]
Arousal: [1 2 0]
Dominance: [1 0 2]
   valence  arousal  dominance
0        2        1          1
1        1        1          0
2        2        2          2
3        2        1          1
4        2        2          2


,psdtheta_1,psdalpha_1,psdbeta_1,psddelta_1,psdgamma_1,psdtheta_2,psdalpha_2,psdbeta_2,psddelta_2,psdgamma_2,...,psdbeta_14,psddelta_14,psdgamma_14,age,gender,participant,video,valence,arousal,dominance
0,-0.150843,-0.150624,-0.166261,-0.151180,-0.243280,-0.145117,-0.143640,-0.119170,-0.146081,-0.184538,...,-0.204449,-0.195448,-0.210802,22,male,1,1,2,1,1
1,-0.128521,-0.129717,-0.155580,-0.127869,-0.025549,-0.105668,-0.108871,-0.168438,-0.103923,-0.033527,...,-0.166154,-0.152931,-0.171532,22,male,1,2,1,1,0
2,-0.103209,-0.103515,-0.122637,-0.103177,-0.176490,-0.109388,-0.111235,-0.140875,-0.108344,-0.111861,...,-0.051510,-0.046362,-0.092479,22,male,1,3,2,2,2
3,-0.181723,-0.184244,-0.230825,-0.180332,-0.178705,-0.113030,-0.114522,-0.139100,-0.112202,-0.126508,...,-0.153138,-0.150298,-0.148628,22,male,1,4,2,1,1
4,-0.043894,-0.042331,-0.032673,-0.044931,-0.149471,-0.148676,-0.152181,-0.201394,-0.146637,-0.124524,...,0.108396,0.073444,-0.135465,22,male,1,5,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,-0.259796,-0.264659,-0.351959,-0.257169,-0.208289,-0.215325,-0.223008,-0.347720,-0.211023,-0.167047,...,-0.049186,-0.053316,-0.117128,25,male,23,14,1,1,1
410,-0.220586,-0.227271,-0.333407,-0.216912,-0.169864,-0.083146,-0.084745,-0.115249,-0.082358,-0.096989,...,0.012865,0.074680,0.144364,25,male,23,15,1,1,1
411,-0.054579,-0.056091,-0.082380,-0.053773,-0.129757,0.014625,0.014887,0.025082,0.014500,0.006592,...,-0.061149,-0.023717,-0.015227,25,male,23,16,1,1,1
412,-0.242060,-0.244057,-0.288320,-0.241022,-0.216108,-0.134908,-0.139699,-0.208673,-0.132119,-0.089472,...,-0.121816,-0.119211,-0.159887,25,male,23,17,1,1,2


In [ ]:
data.isnull().sum().sum()

0

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select columns for normalization (excluding non-numeric columns like 'participant', 'video', etc.)
columns_to_normalize = data.columns.difference(['participant', 'video', 'gender', 'valence', 'arousal', 'dominance'])

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the data and transform the selected columns
data[columns_to_normalize] = scaler.fit_transform(data[columns_to_normalize])
data['gender'].replace({'male': 1, 'female': 0}, inplace=True)
data

,psdtheta_1,psdalpha_1,psdbeta_1,psddelta_1,psdgamma_1,psdtheta_2,psdalpha_2,psdbeta_2,psddelta_2,psdgamma_2,...,psdbeta_14,psddelta_14,psdgamma_14,age,gender,participant,video,valence,arousal,dominance
0,0.027092,0.028112,0.048820,0.026513,0.003533,0.017479,0.018956,0.046369,0.016644,0.002165,...,0.004064,0.003499,0.003304,0.000000,1,1,1,2,1,1
1,0.028463,0.029399,0.049582,0.027944,0.022924,0.020017,0.021200,0.042823,0.019352,0.013740,...,0.005980,0.005674,0.006253,0.000000,1,1,2,1,1,0
2,0.030018,0.031010,0.051931,0.029461,0.009481,0.019778,0.021048,0.044807,0.019068,0.007736,...,0.011716,0.011125,0.012191,0.000000,1,1,3,2,2,2
3,0.025195,0.026044,0.044216,0.024722,0.009284,0.019544,0.020835,0.044935,0.018821,0.006613,...,0.006631,0.005808,0.007973,0.000000,1,1,4,2,1,1
4,0.033662,0.034774,0.058346,0.033038,0.011888,0.017250,0.018405,0.040451,0.016609,0.006765,...,0.019716,0.017253,0.008962,0.000000,1,1,5,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,0.020398,0.021098,0.035578,0.020003,0.006650,0.012963,0.013835,0.029920,0.012473,0.003506,...,0.011832,0.010769,0.010339,0.272727,1,23,14,1,1,1
410,0.022807,0.023398,0.036901,0.022476,0.010072,0.021466,0.022757,0.046652,0.020738,0.008876,...,0.014936,0.017316,0.029980,0.272727,1,23,15,1,1,1
411,0.033005,0.033928,0.054802,0.032495,0.013643,0.027756,0.029186,0.056752,0.026959,0.016814,...,0.011234,0.012283,0.017993,0.272727,1,23,16,1,1,1
412,0.021488,0.022365,0.040116,0.020995,0.005953,0.018136,0.019211,0.039928,0.017541,0.009452,...,0.008198,0.007399,0.007128,0.272727,1,23,17,1,1,2


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score
from sklearn.model_selection import KFold
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.feature_selection import mutual_info_classif
from imblearn.over_sampling import RandomOverSampler

In [ ]:
def Feature_Selection(data, threshold):
    y = data.arousal
    X = data.drop('arousal', axis=1)
    importance = mutual_info_classif(X, y)
    feat_importance = pd.Series(importance, index=X.columns)
    feat_importance = feat_importance.sort_values(ascending=False)
    selected_features = [i for i in feat_importance.index if feat_importance[i] > threshold]
    df_selected_features = data[selected_features + ['arousal']]
    return df_selected_features

# Apply feature selection
EEG_selected_features = Feature_Selection(data, 0.001)

# Fix the copy issue by using .loc
EEG_selected_features.loc[:, "arousal"] = data.arousal

# Handle resampling
X = EEG_selected_features.drop('arousal', axis=1)
y = EEG_selected_features['arousal']
oversampler = RandomOverSampler(random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X, y)


from collections import Counter

# Before oversampling
print("Class Distribution Before Oversampling:")
print(Counter(y))

# After oversampling
print("\nClass Distribution After Oversampling:")
print(Counter(y_resampled))


Class Distribution Before Oversampling:
Counter({1: 212, 2: 181, 0: 21})

Class Distribution After Oversampling:
Counter({1: 212, 2: 212, 0: 212})


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)

# Convert to numpy arrays and ensure the correct data type
X_train = X_train.to_numpy().astype('float32')
X_test = X_test.to_numpy().astype('float32')
y_train = y_train.to_numpy().astype('int32')
y_test = y_test.to_numpy().astype('int32')

# Add a dimension for CNN and RNN input
X_train_cnn = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Reshape for RNN
X_train_rnn = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_rnn = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
!pip install pandas numpy tensorflow scikit-learn imblearn openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_selection import mutual_info_classif
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, Flatten, LSTM, Dropout
from tensorflow.keras.utils import to_categorical
import time
from keras.optimizers import Adam

In [ ]:
# Define deep learning models
def create_cnn(input_shape):
    model = Sequential()
    model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(input_shape, 1)))
    model.add(Conv1D(32, kernel_size=3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_dnn(input_shape):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=(input_shape,)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_rnn(input_shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=(input_shape, 1), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model


def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=20, batch_size=32):
    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
    training_time = time.time() - start_time

    y_pred = (model.predict(X_test) > 0.5).astype("int32")

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    return {
        'history': history,
        'training_time': training_time,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Create and train models
cnn_model = create_cnn(X_train_cnn.shape[1])
dnn_model = create_dnn(X_train.shape[1])
# Create and train RNN model
rnn_model = create_rnn(X_train_rnn.shape[1])



In [ ]:
print("Training CNN...")
cnn_results = train_and_evaluate_model(cnn_model, X_train_cnn, y_train, X_test_cnn, y_test)

print("Training DNN...")
dnn_results = train_and_evaluate_model(dnn_model, X_train, y_train, X_test, y_test)

print("Training RNN...")
rnn_results = train_and_evaluate_model(rnn_model, X_train_rnn, y_train, X_test_rnn, y_test)

# Save the results to an Excel file
results = [
    ["CNN", cnn_results['accuracy'] * 100, cnn_results['precision'] * 100, cnn_results['recall'] * 100, cnn_results['f1'] * 100, cnn_results['training_time']],
    ["DNN", dnn_results['accuracy'] * 100, dnn_results['precision'] * 100, dnn_results['recall'] * 100, dnn_results['f1'] * 100, dnn_results['training_time']],
    ["RNN", rnn_results['accuracy'] * 100, rnn_results['precision'] * 100, rnn_results['recall'] * 100, rnn_results['f1'] * 100, rnn_results['training_time']]
]

df_results = pd.DataFrame(results, columns=['Model', 'Test Accuracy', 'Precision', 'Recall', 'F1 Score', 'Training Time (seconds)'])
output_file = 'Dl_freq_arousal_binary_results.xlsx'
df_results.to_excel(output_file, index=False)

print("Evaluation results saved to:", output_file)

Training CNN...
Epoch 1/20
14/14 [==============================] - 1s 26ms/step - loss: 1.0903 - accuracy: 0.4000 - val_loss: 1.0773 - val_accuracy: 0.4660
Epoch 2/20
14/14 [==============================] - 0s 8ms/step - loss: 1.0721 - accuracy: 0.4360 - val_loss: 1.0844 - val_accuracy: 0.4817
Epoch 3/20
14/14 [==============================] - 0s 8ms/step - loss: 1.0461 - accuracy: 0.4764 - val_loss: 1.0225 - val_accuracy: 0.4764
Epoch 4/20
14/14 [==============================] - 0s 8ms/step - loss: 1.0053 - accuracy: 0.5101 - val_loss: 1.0012 - val_accuracy: 0.3927
Epoch 5/20
14/14 [==============================] - 0s 8ms/step - loss: 0.9815 - accuracy: 0.5011 - val_loss: 0.9343 - val_accuracy: 0.5812
Epoch 6/20
14/14 [==============================] - 0s 9ms/step - loss: 0.9355 - accuracy: 0.5573 - val_loss: 0.9031 - val_accuracy: 0.5916
Epoch 7/20
14/14 [==============================] - 0s 9ms/step - loss: 0.8889 - accuracy: 0.5888 - val_loss: 0.8968 - val_accuracy: 0.5759
Epo

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation results saved to: Dl_freq_arousal_TERNARY_results.xlsx
